In [21]:
import pandas as pd
import numpy as np

In [22]:
fake = pd.read_csv("fake.csv")
true = pd.read_csv("True.csv")
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [23]:
true["lable"]=1
fake["lable"]=0
true.head()

,title,text,subject,date,lable
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [24]:
news = pd.concat([true,fake],axis=0)

Feature Selection From the Data

In [25]:
news=news.drop(columns="title")

In [26]:
news=news.drop(columns="subject")

In [27]:
news=news.drop(columns="date")

In [28]:
news.head()

,text,lable
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [29]:
import re

In [30]:
def wordopt(text):
    text=text.lower()
    text= re.sub(r"https?://\S+|www\.\S+","",text)
    text=re.sub(r'[^\w\s]',"",text)
    text=re.sub(r'<.*?>',"",text)
    text=re.sub(r"\d","",text)
    text=re.sub(r"\n"," ",text)
    return text

In [31]:
news["text"] = news["text"].apply(wordopt)

In [32]:
news["text"]

0        washington reuters  the head of a conservative...
1        washington reuters  transgender people will be...
2        washington reuters  the special counsel invest...
3        washington reuters  trump campaign adviser geo...
4        seattlewashington reuters  president donald tr...
                               ...                        
23476    st century wire says as wire reported earlier ...
23477    st century wire says it s a familiar theme whe...
23478    patrick henningsen  st century wireremember wh...
23479    st century wire says al jazeera america will g...
23480    st century wire says as wire predicted in its ...
Name: text, Length: 44898, dtype: object

In [33]:
x = news['text']
y = news["lable"]

In [34]:
x

0        washington reuters  the head of a conservative...
1        washington reuters  transgender people will be...
2        washington reuters  the special counsel invest...
3        washington reuters  trump campaign adviser geo...
4        seattlewashington reuters  president donald tr...
                               ...                        
23476    st century wire says as wire reported earlier ...
23477    st century wire says it s a familiar theme whe...
23478    patrick henningsen  st century wireremember wh...
23479    st century wire says al jazeera america will g...
23480    st century wire says as wire predicted in its ...
Name: text, Length: 44898, dtype: object

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [37]:
x_train.shape

(31428,)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
vectorzation = TfidfVectorizer()

In [40]:
xv_train = vectorzation.fit_transform(x_train)
xv_test = vectorzation.transform(x_test)

In [41]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2714721 stored elements and shape (13470, 175143)>

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
LR = LogisticRegression()

In [44]:
LR.fit(xv_train,y_train)

LogisticRegression()

In [45]:
pred_lr = LR.predict(xv_test)

In [46]:
LR.score(xv_test,y_test)

0.9898292501855976

In [47]:
from sklearn.tree import DecisionTreeClassifier

In [48]:
DTC = DecisionTreeClassifier()

In [49]:
DTC.fit(xv_train,y_train)

DecisionTreeClassifier()

In [50]:
pred_dtc = DTC.predict(xv_test)

In [51]:
DTC.score(xv_test,y_test)

0.9955456570155902

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [53]:
rfc = RandomForestClassifier()

In [54]:
rfc.fit(xv_train,y_train)

RandomForestClassifier()

In [55]:
predict_rfc = rfc.predict(xv_test)

In [56]:
rfc.score(xv_test,y_test)

0.9900519673348182

In [57]:
from sklearn.ensemble import GradientBoostingClassifier

In [58]:
gbc = GradientBoostingClassifier()

In [59]:
gbc.fit(xv_train,y_train)

GradientBoostingClassifier()

In [60]:
pred_gbc = gbc.predict(xv_test)

In [61]:
gbc.score(xv_test,y_test)

0.9949517446176689

In [62]:
def output(n):
    if n==0:
        return"FAKE NEWS"
    elif n==1:
        return"GENUINE NEWS"

In [76]:
def manualtesting(news):
    testing_news = {"text": news}
    new_det_test = pd.DataFrame([testing_news]) # Important change: Wrap in a list
    new_def_test = new_det_test.copy() # Avoid SettingWithCopyWarning
    new_def_test['text'] = new_def_test['text'].apply(wordopt) # Now this works
    new_x_test = new_def_test["text"]
    new_xv_test = vectorzation.transform(new_x_test)

    pred_lr = LR.predict(new_xv_test)
    pred_gbc = gbc.predict(new_xv_test)
    pred_rfc = rfc.predict(new_xv_test)

    return "\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output(pred_lr[0]), output(pred_gbc[0]), output(pred_rfc[0]))

In [64]:
news_article = str(input())

In [77]:
manualtesting(news_article)

'\nLR Prediction: FAKE NEWS \nGBC Prediction: FAKE NEWS \nRFC Prediction: FAKE NEWS'